In [1]:
using MultiJuMP, JuMP
using Gurobi
using LinearAlgebra
using LightGraphs
if split(pwd(),Base.Filesystem.path_separator)[end] != "CurricularAnalytics.jl"
    cd("../..")
end
println("Current directory: " * pwd())
using Pkg
pkg"activate ."
using CurricularAnalytics
function readfile(file_path)
    open(file_path) do f 
        lines = readlines(f)
        return lines
    end
end

toxicFile = readfile("examples/Notebooks/CourseCombinations_toxic.csv")
synergisticFile = readfile("examples/Notebooks/CourseCombinations_synergistic.csv")
println("Files are read")

comboDict = Dict()
for coursePair in toxicFile[2:end] 
    coursePair = split(coursePair, ",")
    comboDict[replace(coursePair[1], " "=> "")*"_"*replace(coursePair[2], " "=> "")] = parse(Float64,coursePair[9])+1
end
for coursePair in synergisticFile[2:end] 
    coursePair = split(coursePair, ",")
    comboDict[replace(coursePair[1], " "=> "")*"_"*replace(coursePair[2], " "=> "")] = parse(Float64,coursePair[9])+1
end

length(comboDict)

Current directory: /Users/orhanabar/Desktop/CA_Project/CurricularAn/Opt/CurricularAnalytics.jl


┌ Info: Recompiling stale cache file /Users/orhanabar/.julia/compiled/v1.0/CurricularAnalytics/pJfqI.ji for CurricularAnalytics [593ffa3d-269e-5d81-88bc-c3b6809c35a6]
└ @ Base loading.jl:1190


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/915ec9b1c760b4a55fdcd04339125526a31beda4-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/915ec9b1c760b4a55fdcd04339125526a31beda4-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

Files are read


41320

In [12]:
function getVertex(courseID, curric)
    for course in curric.courses
        if course.id == courseID
            return course.vertex_id[curric.id]
        end
    end
    return 0
end
#consequtiveCourses = Dict(1116551982=>2674820998)
#fixedCourses = Dict(502542156=>1, 2071595408=>1)
curric = read_csv("examples/UKY_EE_curric.csv")

Curriculum(3309643389, "University of Kentucky EE Program", "", BS::Degree = 4, semester::System = 0, "26.0101", Course[Course(16242192, Dict(3309643389=>1), "Intro. to Electronics", 3, "EE", "461G", "", "Canonical Name", Dict(2281989961=>pre), LearningOutcome[], Dict{String,Any}()), Course(24551683, Dict(3309643389=>2), "General University Physics", 4, "PHY", "232", "", "Canonical Name", Dict(2865398329=>pre,2193700744=>co), LearningOutcome[], Dict{String,Any}()), Course(109586583, Dict(3309643389=>3), "Supportive Elective", 3, "", "", "", "", Dict{Int64,Requisite}(), LearningOutcome[], Dict{String,Any}()), Course(147960851, Dict(3309643389=>4), "Math/Statistics Elective", 3, "", "", "", "", Dict{Int64,Requisite}(), LearningOutcome[], Dict{String,Any}()), Course(448709314, Dict(3309643389=>5), "Elective EE Laboratory 1", 2, "", "", "", "", Dict{Int64,Requisite}(), LearningOutcome[], Dict{String,Any}()), Course(502542156, Dict(3309643389=>6), "Fundamentals of Engineering Computing", 2,

In [111]:
using CSV
using DataFrames
function remove_empty_lines(file_path)
    if file_path[end-3:end] != ".csv"
        println("Input is not a csv file")
        return false
    end
    temp_file = file_path[1:end-4] * "_temp.csv"
    file = readfile(file_path)
    open(temp_file, "w") do f  
        new_file = ""
        for line in file
            line = replace(line, "\r" => "")
            if length(line) > 0 && !startswith(replace(line,"\""=>""), "#")
                line = line * "\n"
                new_file = new_file * line
            end
        end
        if length(new_file) > 0
            new_file = chop(new_file)
        end
        write(f,new_file)
    end
    return temp_file
end
function csv_line_reader(line::AbstractString, delimeter::Char=',')
    quotes = false
    result = String[]
    item = ""
    for ch in line
        if ch == '"'
            quotes != quotes
        elseif ch == delimeter && !quotes
            push!(result,item)
            item = ""
        else
            item = item * string(ch)
        end
    end
    if length(item) > 0
        push!(result, item)
    end
    return result
end
function read_Opt_Config(file_path)
    file_path = remove_empty_lines(file_path)
    if typeof(file_path) == Bool && !file_path
        return false
    end
    consequtiveCourses = Dict()
    fixedCourses = Dict()
    termRange = Dict()
    header = 0
    termCount = 0
    min_credits_per_term = 0
    max_credits_per_term = 0
    obj_order = []
    open(file_path) do csv_file        
        read_line = csv_line_reader(readline(csv_file), ',')
        header += 1
        if read_line[1] == "Term Count"
            termCount = parse(Int, read_line[2])
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
        else
            println("First line of config file must contain term count")
        end
        if read_line[1] == "Min Credit"
            min_credits_per_term = parse(Int, read_line[2])
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
        else
            println("First line of config file must contain Min Credit")
        end
        if read_line[1] == "Max Credit"
            max_credits_per_term = parse(Int, read_line[2])
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
        else
            println("First line of config file must contain Max Credit")
        end
        if read_line[1] == "Objective Order"
            obj_order = parse.(Int, split(read_line[2],";"))           
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
        else
            println("First line of config file must contain Max Credit")
        end
        if read_line[1] == "Fixed Terms"
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
            if read_line[1] != "Course ID" || read_line[2] != "Term"
                println("Error detected with fixed courses headers.")
                return false
            end
            read_line = csv_line_reader(readline(csv_file), ',')
            course_count = 0
            while length(read_line) > 0 && read_line[1] != "Consecutive Terms" && read_line[1] != "Term Range" !startswith(read_line[1], "#")
                if length(read_line[1]) == 0 || length(read_line[1]) == 0
                    println("Each course must have a Course ID and Term number")
                    return false
                end
                course_count += 1
                read_line = csv_line_reader(readline(csv_file), ',')
            end
            df_fixedCourses = CSV.File(file_path, header=header, limit=course_count) |> DataFrame
            header += course_count+1
            for row in DataFrames.eachrow(df_fixedCourses)
                fixedCourses[row[Symbol("Course ID")]] = row[Symbol("Term")]
            end
        end
        if read_line[1] == "Consecutive Terms"
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
            if read_line[1] != "Prior Course ID" || read_line[2] != "Next Course ID"
                println("Error detected with Consecutive Terms headers.")
                return false
            end
            consecutivePairCount = 0
            read_line = csv_line_reader(readline(csv_file), ',')
            while length(read_line) > 0 && read_line[1] != "Term Range" !startswith(read_line[1], "#")
                if length(read_line[1]) == 0 || length(read_line[1]) == 0
                    println("Each pair must have two Course IDs")
                    return false
                end
                consecutivePairCount += 1
                read_line = csv_line_reader(readline(csv_file), ',')
            end
            df_consecutivePair = CSV.File(file_path, header=header, limit=consecutivePairCount) |> DataFrame
            header += consecutivePairCount+1
            for row in DataFrames.eachrow(df_consecutivePair)
                consequtiveCourses[row[Symbol("Prior Course ID")]] = row[Symbol("Next Course ID")]
            end
        end
        if read_line[1] == "Term Range"
            read_line = csv_line_reader(readline(csv_file), ',')
            header += 1
            if read_line[1] != "Course Id" || read_line[2] != "Min Term" || read_line[3] != "Max Term" 
                println("Error detected with Consecutive Terms headers.")
                return false
            end
            termRangeCount = 0
            read_line = csv_line_reader(readline(csv_file), ',')
            while length(read_line) > 0 && read_line[1] != "Term Range" !startswith(read_line[1], "#")
                if length(read_line[1]) == 0 || length(read_line[1]) == 0
                    println("Each pair must have two Course IDs")
                    return false
                end
                termRangeCount += 1
                read_line = csv_line_reader(readline(csv_file), ',')
            end
            df_termRange = CSV.File(file_path, header=header, limit=termRangeCount) |> DataFrame
            header += termRangeCount+1
            for row in DataFrames.eachrow(df_termRange)
                termRange[row[Symbol("Course Id")]] = (row[Symbol("Min Term")], row[Symbol("Max Term")])
            end
        end
        
    end
    return consequtiveCourses, fixedCourses, termRange, termCount, min_credits_per_term, max_credits_per_term,obj_order
end

read_Opt_Config (generic function with 1 method)

In [109]:
a = [5,6,7]
b = [3,2,1]
a[b]

3-element Array{Int64,1}:
 7
 6
 5

In [112]:
consequtiveCourses, fixedCourses, termRange, termCount, min_credits_per_term, max_credits_per_term, obj_order = 
    read_Opt_Config("examples/Notebooks/opt_Configurations.csv")
if length(obj_order) == 1
    print("call single objective")
end
print(obj_order)
println("Number of courses in curriculum: "*string(length(curric.courses)))
println("Total credit hours: "*string(total_credits(curric)))
#m = Model(solver = GurobiSolver())
m = multi_model(solver = GurobiSolver(), linear = true)
courses = curric.courses
c_count = length(courses)
toxicity_scores = zeros((c_count, c_count))
for course in courses
    for innerCourse in courses
        if course != innerCourse 
            if course.prefix*course.num*"_"*innerCourse.prefix*innerCourse.num in keys(comboDict)
                toxicity_scores[course.vertex_id[curric.id],innerCourse.vertex_id[curric.id]] = comboDict[course.prefix*course.num*"_"*innerCourse.prefix*innerCourse.num]
            end
        end
    end
end
credit = [c.credit_hours for c in curric.courses]

mask = [i for i in 1:termCount]
@variable(m, x[1:c_count, 1:termCount], Bin)
ts = []
for j=1:termCount
    push!(ts, sum((toxicity_scores .* x[:,j]) .* x[:,j]'))
end
terms = [sum(dot(x[k,:],mask)) for k = 1:c_count]
vertex_map = Dict{Int,Int}(c.id => c.vertex_id[curric.id] for c in curric.courses)
total_credit_term = [sum(dot(credit,x[:,j])) for j=1:termCount]
@variable(m, 0 <= y[1:termCount] <= max_credits_per_term)
@constraints m begin
    #output must include all courses once
    tot[i=1:c_count], sum(x[i,:]) == 1
    #Each term must include more or equal than min credit and less or equal than max credit allowed for a term
    term_upper[j=1:termCount], sum(dot(credit,x[:,j])) <= max_credits_per_term
    term_lower[j=1:termCount], sum(dot(credit,x[:,j])) >= min_credits_per_term
    abs_val[i=2:termCount], y[i] >= total_credit_term[i]-total_credit_term[i-1]
    abs_val2[i=2:termCount], y[i] >= -(total_credit_term[i]-total_credit_term[i-1])
end
for c in courses
    for req in c.requisites
        if req[2] == pre
            @constraint(m, sum(dot(x[vertex_map[req[1]],:],mask)) <= (sum(dot(x[c.vertex_id[curric.id],:],mask))-1))
        elseif req[2] == co
            @constraint(m, sum(dot(x[vertex_map[req[1]],:],mask)) <= (sum(dot(x[c.vertex_id[curric.id],:],mask))))
        elseif req[2] == strict_co
            @constraint(m, sum(dot(x[vertex_map[req[1]],:],mask)) == (sum(dot(x[c.vertex_id[curric.id],:],mask))))
        else
            println("req type error")
        end
    end   
end
if length(keys(fixedCourses)) > 0
    for courseID in keys(fixedCourses)
        vID = getVertex(courseID, curric)
        if vID != 0
            @constraint(m, x[vID,fixedCourses[courseID]] >= 1)
        else
            println("Vertex ID cannot be found for course: "* courseName)
        end
    end
end
if length(keys(consequtiveCourse)) > 0
    for (first, second) in consequtiveCourse
        vID_first = getVertex(first, curric)
        vID_second = getVertex(second, curric)
        if vID_first != 0 && vID_second != 0
            @constraint(m, sum(dot(x[vID_second,:],mask)) - sum(dot(x[vID_first,:],mask)) <= 1)
            @constraint(m, sum(dot(x[vID_second,:],mask)) - sum(dot(x[vID_first,:],mask)) >= 1)
        else
            println("Vertex ID cannot be found for course: "* first * " or " * second)
        end
    end
end
if length(keys(termRange)) > 0
    for (courseID,(lowTerm, highTerm)) in termRange
        vID_Course = getVertex(courseID, curric)
        if vID_Course != 0
            @constraint(m, sum(dot(x[vID_Course,:],mask)) >= lowTerm)
            @constraint(m, sum(dot(x[vID_Course,:],mask)) <= highTerm)
        end
    end
end

total_distance = []
for edge in collect(edges(curric.graph))
    push!(total_distance, sum(dot(x[dst(edge),:],mask)) - sum(dot(x[src(edge),:],mask)))
end

#println(ts[1])
exp1 = @expression(m, sum(ts[:]))
exp2 = @expression(m, sum(y[:]))
exp3 = @expression(m, sum(total_distance[:]))

obj1 = SingleObjective(exp1, sense = :Min)
obj2 = SingleObjective(exp2, sense = :Min)
obj3 = SingleObjective(exp3, sense = :Min)
multim = get_multidata(m)
objectives = [obj1, obj2, obj3]
multim.objectives = objectives[obj_order]

#solve(m)
#@objective(m, Min, sum(total_distance[:]))
#println("started to solve")
status = solve(m)
if status == :Optimal
    output = getvalue(x)
    println(sum(getvalue(y)))
    println(sum(getvalue(ts)))
    println(sum(getvalue(total_distance)))
    #println(ts)
    optimal_terms = Term[]
    for j=1:termCount
        if sum(dot(credit,output[:,j])) > 0 
            term = Course[]
            for course in 1:length(courses)
                if round(output[course,j]) == 1
                    push!(term, courses[course])
                end 
            end
            push!(optimal_terms, Term(term))
        end
    end
    dp = DegreePlan("", curric, optimal_terms)
    visualize(dp, notebook=true)
else
    println("not optimal")
end

[2, 1, 3]Number of courses in curriculum: 43
Total credit hours: 131
Academic license - for non-commercial use only
Optimize a model with 109 rows, 352 columns and 2765 nonzeros
Variable types: 8 continuous, 344 integer (344 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 17 rows and 97 columns
Presolve time: 0.01s
Presolved: 92 rows, 255 columns, 883 nonzeros
Variable types: 0 continuous, 255 integer (241 binary)
Found heuristic solution: objective 25.0000000

Root relaxation: objective 0.000000e+00, 70 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   21   25.00000    0.00000   100%     -    0s
H    0     0                       8.0000000    0.00000   100%     -    0s
H    0     0              

1.0
17.006277179378415
62.0


WebIO.Scope("scope-aa7d5ad7-f994-4beb-bfda-ed32a8e4082b", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src=>"http://localhost:8156",:id=>"curriculum",:events=>Dict(:load=>JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"University of Kentucky EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective EE Laboratory 1\",\"id\":448709314,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 102\",\"metrics\":{\"centrality\":0,\"complexity\":6.0,\"blocking factor\":3,\"delay factor\":3.0},\"nameSub\":\"Fundamentals of Engineering Computing\",\"id\":502542156,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"UK Core - Global Dynamics\",\"id\":819264340,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CIS/WRD 110\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Composition and Communication I\",\"id\":858428248,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"EGR 101\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Engineering Exploration I\",\"id\":2071595408,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective EE Laboratory 2\",\"id\":3008452217,\"credits\":2,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MA 113\",\"metrics\":{\"centrality\":0,\"complexity\":23.0,\"blocking factor\":15,\"delay factor\":8.0},\"nameSub\":\"Calculus I\",\"id\":3389674827,\"credits\":4,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Supportive Elective\",\"id\":109586583,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Engineering/Science Elective 2\",\"id\":1227973603,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2865398329,\"type\":\"coreq\"}],\"name\":\"PHY 231\",\"metrics\":{\"centrality\":62,\"complexity\":16.0,\"blocking factor\":10,\"delay factor\":6.0},\"nameSub\":\"General University Physics\",\"id\":2865398329,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":3389674827,\"target_id\":2966989896,\"type\":\"prereq\"}],\"name\":\"MA 114\",\"metrics\":{\"centrality\":102,\"complexity\":19.0,\"blocking factor\":11,\"delay factor\":8.0},\"nameSub\":\"Calculus II\",\"id\":2966989896,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"EE Technical Elective 4\",\"id\":3141179749,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":2865398329,\"target_id\":24551683,\"type\":\"prereq\"},{\"source_id\":2193700744,\"target_id\":24551683,\"type\":\"coreq\"}],\"name\":\"PHY 232\",\"metrics\":{\"centrality\":121,\"complexity\":15.0,\"blocking factor\":7,\"delay factor\":8.0},\"nameSub\":\"General University Physics\",\"id\":24551683,\"credits\":4,\"nameCanonical\":\"Canonical Name\"},{\"curriculum_requisites\":[],\"name\":\" \",

In [ ]:
function getVertex(courses, courseName, curricID)
    for course in courses
        if course.prefix*course.num == courseName
            return course.vertex_id[curricID]
        end
    end
    return 0
end
consequtiveCourse = Dict("EE490"=>"EE491")
fixedCourses = Dict("EGR102"=>1, "EGR101"=>1)
curric = read_csv("examples/UKY_EE_curric.csv")
max_terms = 8
max_credits_per_term = 19
min_credits_per_term = 1
println("Number of courses in curriculum: "*string(length(curric.courses)))
println("Total credit hours: "*string(total_credits(curric)))
m = vModel(solver = GurobiSolver())
#m = multi_model(solver = GurobiSolver(), linear = true)
courses = curric.courses
c_count = length(courses)
toxicity_scores = zeros((c_count, c_count))
for course in courses
    for innerCourse in courses
        if course != innerCourse 
            if course.prefix*course.num*"_"*innerCourse.prefix*innerCourse.num in keys(comboDict)
                toxicity_scores[course.vertex_id[curric.id],innerCourse.vertex_id[curric.id]] = comboDict[course.prefix*course.num*"_"*innerCourse.prefix*innerCourse.num]
            end
        end
    end
end
credit = [c.credit_hours for c in curric.courses]

mask = [i for i in 1:max_terms]
@variable(m, x[1:c_count, 1:max_terms], Bin)
ts = []
for j=1:max_terms
    push!(ts, sum((toxicity_scores .* x[:,j]) .* x[:,j]'))
end
terms = [sum(dot(x[k,:],mask)) for k = 1:c_count]
vertex_map = Dict{Int,Int}(c.id => c.vertex_id[curric.id] for c in curric.courses)
total_credit_term = [sum(dot(credit,x[:,j])) for j=1:max_terms]
@variable(m, 0 <= y[1:max_terms] <= max_credits_per_term)
@constraints m begin
    #output must include all courses once
    tot[i=1:c_count], sum(x[i,:]) == 1
    #Each term must include more or equal than min credit and less or equal than max credit allowed for a term
    term_upper[j=1:max_terms], sum(dot(credit,x[:,j])) <= max_credits_per_term
    term_lower[j=1:max_terms], sum(dot(credit,x[:,j])) >= min_credits_per_term
    abs_val[i=2:max_terms], y[i] >= total_credit_term[i]-total_credit_term[i-1]
    abs_val2[i=2:max_terms], y[i] >= -(total_credit_term[i]-total_credit_term[i-1])
end
for c in courses
    for req in c.requisites
        if req[2] == pre
            @constraint(m, sum(dot(x[vertex_map[req[1]],:],mask)) <= (sum(dot(x[c.vertex_id[curric.id],:],mask))-1))
        elseif req[2] == co
            @constraint(m, sum(dot(x[vertex_map[req[1]],:],mask)) <= (sum(dot(x[c.vertex_id[curric.id],:],mask))))
        elseif req[2] == strict_co
            @constraint(m, sum(dot(x[vertex_map[req[1]],:],mask)) == (sum(dot(x[c.vertex_id[curric.id],:],mask))))
        else
            println("req type error")
        end
    end   
end
if length(keys(fixedCourses)) > 0
    for courseName in keys(fixedCourses)
        vID = getVertex(courses, courseName, curric.id)
        if vID != 0
            @constraint(m, x[vID,fixedCourses[courseName]] >= 1)
        else
            println("Vertex ID cannot be found for course: "* courseName)
        end
        #println(getVertex(courses, courseName, curric.id))
    end
end
if length(keys(consequtiveCourse)) > 0
    for (first, second) in consequtiveCourse
        vID_first = getVertex(courses, first, curric.id)
        vID_second = getVertex(courses, second, curric.id)
        if vID_first != 0 && vID_second != 0
            @constraint(m, sum(dot(x[vID_first,:],mask)) >= 7)
            @constraint(m, sum(dot(x[vID_second,:],mask)) - sum(dot(x[vID_first,:],mask)) <= 1)
            @constraint(m, sum(dot(x[vID_second,:],mask)) - sum(dot(x[vID_first,:],mask)) >= 1)
        else
            println("Vertex ID cannot be found for course: "* first * " or " * second)
        end
    end
end
total_distance = []
for edge in collect(edges(curric.graph))
    push!(total_distance, sum(dot(x[dst(edge),:],mask)) - sum(dot(x[src(edge),:],mask)))
end

#println(ts[1])
exp1 = @expression(m, sum(ts[:]))
exp2 = @expression(m, sum(y[:]))
exp3 = @expression(m, sum(total_distance[:]))

obj1 = SingleObjective(exp1, sense = :Min)
obj2 = SingleObjective(exp2, sense = :Min)
obj3 = SingleObjective(exp3, sense = :Min)
multim = get_multidata(m)
multim.objectives = [obj2, obj1, obj3]

#solve(m)
#@objective(m, Min, sum(ts[:]))
println("started to solve")
status = solve(m)
if status == :Optimal
    output = getvalue(x)
    println(sum(getvalue(y)))
    println(sum(getvalue(ts)))
    println(sum(getvalue(total_distance)))
    #println(ts)
    optimal_terms = Term[]
    for j=1:max_terms
        if sum(dot(credit,output[:,j])) > 0 
            term = Course[]
            for course in 1:length(courses)
                if round(output[course,j]) == 1
                    push!(term, courses[course])
                end 
            end
            push!(optimal_terms, Term(term))
        end
    end
    dp = DegreePlan("", curric, optimal_terms)
    visualize(dp, notebook=true)
else
    println("not optimal")
end